# 🚀 **DevOps Command Line Assistant using `agno` and OpenAI**


---


Welcome! This notebook walks you through building a DevOps Assistant that interprets natural language commands to manage infrastructure tasks like Kubernetes, CI/CD, Linux services, and Terraform—all routed through intelligent agents.


---



## 📦 **1. Install Dependencies**
We start by installing the `agno` framework, which enables multi-agent workflows with AI models.

In [2]:
!pip install agno -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.6/616.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 15.6 MB/s eta 0:00:00


## 🔐 **2. Set Up Environment Variables**
To use OpenAI's models, you need to provide your API key. For security, set it via an environment variable.

In [3]:
import os
os.environ['OPENAI_API_KEY'] = 'Enter your open ai key here"

## 🧠 **3. Import Required Libraries**
Let's import the core components needed to define agents and teams using `agno`.

In [4]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.team.team import Team

## 🛠️ **4. Define Individual DevOps Agents**
Each agent specializes in a particular area: Linux operations, Kubernetes, Terraform, or CI/CD.

In [30]:

# 🐧 LinuxOps Agent
# Handles common Linux command-line operations such as restarting services, checking logs, and system diagnostics.
linux_agent = Agent(
    name='LinuxOps Agent',
    role='Handles Linux command line tasks such as restarting services and checking logs.',
    model=OpenAIChat(id='gpt-4o'),
)

# ☸️ Kubernetes Agent
# Specializes in managing Kubernetes clusters via kubectl — including inspecting pod status and managing deployments.
k8s_agent = Agent(
    name='Kubernetes Agent',
    role='Manages Kubernetes clusters using kubectl for tasks like checking pod statuses and managing deployments.',
    model=OpenAIChat(id='gpt-4o'),
)

# ⚙️ Terraform Agent
# Executes infrastructure-as-code operations using Terraform — applies, plans, state management, and validation.
terraform_agent = Agent(
    name='Terraform Agent',
    role='Manages infrastructure using Terraform commands.',
    model=OpenAIChat(id='gpt-4o'),
)

# 🔄 CI/CD Agent
# Handles automation around continuous integration and deployment — like triggering builds and checking pipeline status.
cicd_agent = Agent(
    name='CI/CD Agent',
    role='Handles CI/CD tasks, Trigger CI/CD pipelines like Jenkins builds, such as triggering builds and checking pipeline statuses.',
    model=OpenAIChat(id='gpt-4o'),
)


## 👥 **5. Build the DevOps Team**
The team acts as a router. Based on the type of task, it dispatches to the right agent.

In [59]:

# This team intelligently routes infrastructure-related tasks to the right domain-specific agent (Linux, Kubernetes, Terraform, CI/CD).
devops_team = Team(
    name='DevOps Command Assistant',

    # 🧭 Routing mode: Automatically selects the most appropriate agent based on task context.
    mode='route',

    # 🔧 Default model for routing logic
    model=OpenAIChat(id='gpt-4o'),

    # 👥 Members: Specialized agents for different DevOps domains
    members=[linux_agent, k8s_agent, terraform_agent, cicd_agent],

    # 💬 Whether to show tool call traces in the output
    show_tool_calls=False,

    # 📝 Enable Markdown formatting in responses
    markdown=True,

    # 📘 Description of the team's purpose
    description='Routes infrastructure tasks to the appropriate specialist agent.',

    # 📜 Routing instructions to guide how the assistant delegates tasks
    instructions=[
        'Determine the nature of the task: Linux, Kubernetes, Terraform, or CI/CD.',
        'Route the task to the appropriate agent.',
        'If the task is ambiguous, ask the user for clarification.'
    ],

    # 👀 Hides individual agent responses unless needed for debugging
    show_members_responses= True
)


## 🧪 **6. Try Sample Prompts**
Now let's see the assistant in action with a few natural language commands.

In [7]:
# Restart a Linux service
devops_team.print_response('Restart the nginx service on production-server',stream = True)


Output()

In [8]:
# Check Kubernetes pods
devops_team.print_response('Get all pods in kube-system namespace',stream = True)



Output()

In [23]:
# Apply Terraform plan
devops_team.print_response('print_response terraform apply on the staging environment',stream = True)


Output()

In [32]:
# Trigger CI/CD pipeline
devops_team.print_response('Trigger the nightly Jenkins build',stream = True)

Output()

# Evaluation Using Llumo AI

In [74]:
user_query = "Is there a way to cancel a Jenkins job?"
response = devops_team.run(user_query)
response = response.member_responses[0].content

In [75]:
# Code to Evaluate the output
User_Query= user_query
llm_response = response
LLUMO_API_KEY = "Enter Your Llumo Api Here"
inputs = {}
import requests
# Define the endpoint, headers, and payload
LLUMO_ENDPOINT = "https://app.llumo.ai/api/create-eval-analytics"
headers = {
    "Authorization": f"Bearer {LLUMO_API_KEY}", # Replace with your LLumo API key it will look like this "Bearer A1B2C3"
    "Content-Type": "application/json"
}
payload = {
    "prompt": User_Query,
    "input": inputs,
    "output": llm_response,
    "analytics": ["Confidence"] # ANALYTICS NAME are Confidence,Clarity,Context.....etc.
}
# Make the API request
response = requests.post(LLUMO_ENDPOINT, json=payload, headers=headers)
try:
    result = response.json()  # Parse the JSON response
    print("statusCode : ", result['data']['statusCode'])
    print("message : ",result['data']['message'])
    # Extract the 'data' part
    data = result.get('data', {})
    print("Analytics:", data)
    # Return the data and a success flag

except Exception as e:
  print(e)


statusCode :  200
message :  SUCCESS
Analytics: {'data': '{"analyticsScore": {"*definition 1: the output should provide a comprehensive guide on how to cancel a jenkins job, covering both the web ui and command-line interface methods.  it should include clear, step-by-step instructions, address potential issues (like permissions), and offer best practices (like using api tokens).": 90, "*definition 2: the response must accurately describe at least two methods for canceling a jenkins job.  it should mention the use of a web interface and a command-line tool. the explanation of each method should be sufficient for a user to perform the cancellation.": 85, "*definition 3:  the output should correctly explain how to cancel a jenkins job using the jenkins cli. it should include the correct command syntax and mention the need for authentication.": 95, "*definition 4: the response should be concise and to the point, directly answering the prompt\'s question about canceling a jenkins job.  it 

## **🌐 Bonus: Swappable Models**
You can swap `OpenAIChat` with `Gemini`, `DeepSeek`, or others by just changing the model class while keeping the logic intact.